<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://drive.google.com/uc?export=view&id=1LqkEgbpZj8A99Y9T59eBp9fEIZbpg6P2" alt="Snowflake Snowpark Classroom" style="width: 800px">
</div>

# Classroom 1.4 - Table Operations Using Snowflake Snowpark Python API 

In this notebook, you will learn how to perform table operations using Snowflake Snowpark API for Python

## Learning Objectives

By the end of this classroom, you should be able to:
- Perform Delete, Merge, Update Operations on a table

In [2]:
from assets.config import connection_builder
session = connection_builder()

## Section 1 - Understanding Snowpark Session Table Object and it's different methods

snowflake.snowpark.Table represents a lazily-evaluated Table.

You can create a Table object by calling [Session.table()](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.Table) with the name of the table in Snowflake.


Methods of Snowpark Table Object:

| Method | Description |
|--|--|
| delete() | Deletes rows in a Table and returns a DeleteResult, representing the number of rows deleted. |
| drop_table() | Drops the table from the Snowflake database. |
| merge() | Merges this Table with DataFrame source on the specified join expression and a list of matched or not-matched clauses, and returns a MergeResult, representing the number of rows inserted, updated and deleted by this merge action. |
| update() | Updates rows in the Table with specified assignments and returns a UpdateResult, representing the number of rows modified and the number of multi-joined rows modified.| 


In [3]:
# Performing delete() operations on the temporary table

from snowflake.snowpark.types import StructField, StructType, IntegerType, StringType

df_schema = StructType([StructField('username',StringType()),StructField('age',IntegerType()),StructField('email_id',StringType()) ])
df_dataset = [['divyansh',24,'divyansh@abc.com'],['raghav',29,'raghav@abc.in'],['Aman',34,'aman@xyz.ca'],['Leo',22,'leo@abc.com']]
df = session.createDataFrame(df_dataset,df_schema)

src_schema = StructType([StructField('age',IntegerType())])
src_dataset = [[35],[22],[29],[23]]
src_df = session.create_dataframe(src_dataset,src_schema) 



print('----------------- Original Data -----------------')
df.write.save_as_table('test_table',mode='overwrite',table_type='temporary')
session.table('test_table').show()

print('----------------- Delete Operation with no condition -----------------')
table_0 = session.table('test_table').delete()
session.table('test_table').show()

print('----------------- Delete Operation with predicate condition -----------------')
df.write.save_as_table('test_table',mode='overwrite',table_type='temporary')
table_1 = session.table('test_table')
table_1.delete(table_1["age"]==24)
session.table('test_table').show()


print('----------------- Delete Operation with dataset -----------------')
df.write.save_as_table('test_table',mode='overwrite',table_type='temporary')
table_2 = session.table('test_table')
table_2.delete(table_2["age"]==src_df.age,src_df)
session.table('test_table').show()

----------------- Original Data -----------------
-----------------------------------------
|"USERNAME"  |"AGE"  |"EMAIL_ID"        |
-----------------------------------------
|divyansh    |24     |divyansh@abc.com  |
|raghav      |29     |raghav@abc.in     |
|Aman        |34     |aman@xyz.ca       |
|Leo         |22     |leo@abc.com       |
-----------------------------------------

----------------- Delete Operation with no condition -----------------
-----------------------------------
|"USERNAME"  |"AGE"  |"EMAIL_ID"  |
-----------------------------------
|            |       |            |
-----------------------------------

----------------- Delete Operation with predicate condition -----------------
--------------------------------------
|"USERNAME"  |"AGE"  |"EMAIL_ID"     |
--------------------------------------
|raghav      |29     |raghav@abc.in  |
|Aman        |34     |aman@xyz.ca    |
|Leo         |22     |leo@abc.com    |
--------------------------------------

---------

In [4]:
# Understanding Various Drop Methods on the table object.

df_schema = StructType([StructField('username',StringType()),StructField('age',IntegerType()),StructField('email_id',StringType()) ])
df_dataset = [['divyansh',24,'divyansh@abc.com'],['raghav',29,'raghav@abc.in'],['Aman',34,'aman@xyz.ca'],['Leo',22,'leo@abc.com'],['raghav',29,'raghav@abc.in'],['raghav',29,'raghav@abc.in'],['Aman',34,None],[None,None,None]]
df = session.createDataFrame(df_dataset,df_schema)
df.write.save_as_table('test_table',mode='overwrite',table_type='temporary')

session.table('test_table').drop_table()

In [5]:
# Understanding Merge Operations on the Table Object
from snowflake.snowpark.functions import when_matched, when_not_matched
from snowflake.snowpark.types import StructField, StructType, IntegerType, StringType

tgt_schema = StructType([StructField('username',StringType()),StructField('age',IntegerType()),StructField('email_id',StringType()) ])
tgt_dataset = [['divyansh',24,'divyansh@abc.com'],['raghav',29,'raghav@abc.in'],['Aman',34,'aman@xyz.ca'],['Leo',22,'leo@abc.com']]
tgt_df = session.createDataFrame(tgt_dataset,tgt_schema)
tgt_df.write.save_as_table('target_table',mode='overwrite',table_type='temporary')

src_schema = StructType([StructField('username',StringType()),StructField('age',IntegerType()),StructField('email_id',StringType()) ])
src_dataset = [['divyansh',24,'divyansh@abc.com'],['raghav',22,'raghav@def.ca'],['Aman',36,'aman@google.ca'],['Paul',22,'Paul@abc.com']]
src_df = session.createDataFrame(src_dataset,src_schema)
src_df.write.save_as_table('source_table',mode='overwrite',table_type='temporary')

src_table = session.table('source_table')
tgt_table = session.table('target_table')

print('----------------- Source Table Data -----------------')
src_table.show()
print('----------------- Target Table Data -----------------')
tgt_table.show()

tgt_table.merge(src_table,
                (tgt_table['username']==src_table['username']),
                [when_matched().update({'age':src_table['age'], 'email_id':src_table['email_id']}),
                 when_not_matched().insert({'username':src_table['username'],'age':src_table['age'],'email_id':src_table['email_id']})])

print('----------------- Target Table Data After Merge Statement -----------------')
tgt_table.show()



----------------- Source Table Data -----------------
-----------------------------------------
|"USERNAME"  |"AGE"  |"EMAIL_ID"        |
-----------------------------------------
|divyansh    |24     |divyansh@abc.com  |
|raghav      |22     |raghav@def.ca     |
|Aman        |36     |aman@google.ca    |
|Paul        |22     |Paul@abc.com      |
-----------------------------------------

----------------- Target Table Data -----------------
-----------------------------------------
|"USERNAME"  |"AGE"  |"EMAIL_ID"        |
-----------------------------------------
|divyansh    |24     |divyansh@abc.com  |
|raghav      |29     |raghav@abc.in     |
|Aman        |34     |aman@xyz.ca       |
|Leo         |22     |leo@abc.com       |
-----------------------------------------

----------------- Target Table Data After Merge Statement -----------------
-----------------------------------------
|"USERNAME"  |"AGE"  |"EMAIL_ID"        |
-----------------------------------------
|Paul        |22

In [6]:
# Understanding Update Operations on the Table Object

tgt_schema = StructType([StructField('username',StringType()),StructField('age',IntegerType()),StructField('email_id',StringType()) ])
tgt_dataset = [['divyansh',24,'divyansh@abc.com'],['raghav',29,'raghav@abc.in'],['Aman',34,'aman@xyz.ca'],['Leo',22,'leo@abc.com'],['divyansh',18,'divyansh@xyz.com']]
tgt_df = session.createDataFrame(tgt_dataset,tgt_schema)
tgt_df.write.save_as_table('target_table',mode='overwrite',table_type='temporary')
tgt_table = session.table('target_table')

print('----------------- Target Table Data -----------------')
tgt_table.show()

tgt_table.update({'age' : 0},(tgt_table['username']=='divyansh') | ((tgt_table['age']>25) & (tgt_table['age']<30)))

print('----------------- Target Table Data After Update -----------------')
tgt_table.show()

----------------- Target Table Data -----------------
-----------------------------------------
|"USERNAME"  |"AGE"  |"EMAIL_ID"        |
-----------------------------------------
|divyansh    |24     |divyansh@abc.com  |
|raghav      |29     |raghav@abc.in     |
|Aman        |34     |aman@xyz.ca       |
|Leo         |22     |leo@abc.com       |
|divyansh    |18     |divyansh@xyz.com  |
-----------------------------------------

----------------- Target Table Data After Update -----------------
-----------------------------------------
|"USERNAME"  |"AGE"  |"EMAIL_ID"        |
-----------------------------------------
|Aman        |34     |aman@xyz.ca       |
|Leo         |22     |leo@abc.com       |
|divyansh    |0      |divyansh@abc.com  |
|raghav      |0      |raghav@abc.in     |
|divyansh    |0      |divyansh@xyz.com  |
-----------------------------------------



: 